<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/037_JW_Modular_AgentDesign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🧱 Building a Simple Agent Framework

We are designing our agents in terms of **GAME**. Ideally, we want our code to reflect how we design the agent, allowing for easy translation from concept to implementation. The **GAME components** (Goals, Actions, Memory, Environment) are what change from one agent to another, while the core loop remains stable.

Though this may add complexity initially, it promotes long-term flexibility and reusability.

---

## 🎯 G - Goals Implementation

Define **what** the agent is trying to accomplish and **how** it should do it:

```python
from dataclasses import dataclass

@dataclass(frozen=True)
class Goal:
    priority: int
    name: str
    description: str
```

Example:

```python
file_management_goal = Goal(
    priority=1,
    name="file_management",
    description="""Manage files in the current directory by:
    1. Listing files when needed
    2. Reading file contents when needed
    3. Searching within files when information is required
    4. Providing helpful explanations about file contents"""
)
```

---

## 🛠️ A - Actions Implementation with JSON Schemas

Encapsulate actions as objects to improve modularity and reuse:

```python
class Action:
    def __init__(self, name, function, description, parameters, terminal=False):
        self.name = name
        self.function = function
        self.description = description
        self.terminal = terminal
        self.parameters = parameters

    def execute(self, **args):
        return self.function(**args)
```

Register actions using an **ActionRegistry**:

```python
class ActionRegistry:
    def __init__(self):
        self.actions = {}

    def register(self, action: Action):
        self.actions[action.name] = action

    def get_action(self, name: str):
        return self.actions.get(name)

    def get_actions(self):
        return list(self.actions.values())
```

### Example Actions:

```python
def list_files():
    return os.listdir('.')

def read_file(file_name):
    with open(file_name, 'r') as f:
        return f.read()

def search_in_file(file_name, search_term):
    results = []
    with open(file_name, 'r') as f:
        for i, line in enumerate(f.readlines()):
            if search_term in line:
                results.append((i+1, line.strip()))
    return results
```

### Register Actions:

```python
registry = ActionRegistry()

registry.register(Action(
    name="list_files",
    function=list_files,
    description="List all files in the current directory",
    parameters={"type": "object", "properties": {}, "required": []}
))

# ... Register other actions similarly
```

---

## 🧠 M - Memory Implementation

Track what happens across loop iterations:

```python
class Memory:
    def __init__(self):
        self.items = []

    def add_memory(self, memory: dict):
        self.items.append(memory)

    def get_memories(self, limit=None):
        return self.items[:limit]
```

✅ Abstracting memory logic enables future upgrades (e.g., DB, vector search) without rewriting the agent loop.

---

## 🌐 E - Environment Implementation

Encapsulates how the agent interacts with the external world:

```python
class Environment:
    def execute_action(self, action: Action, args: dict) -> dict:
        try:
            result = action.execute(**args)
            return self.format_result(result)
        except Exception as e:
            return {
                "tool_executed": False,
                "error": str(e),
                "traceback": traceback.format_exc()
            }

    def format_result(self, result: Any) -> dict:
        return {
            "tool_executed": True,
            "result": result,
            "timestamp": time.strftime("%Y-%m-%dT%H:%M:%S%z")
        }
```

✅ Keeps the agent loop clean and lets us change how actions are executed without needing new logic in the loop.




##Action Class is Based on OpenAI Tool API requirments

Action Class is shaped that way **on purpose**. Its fields line up with OpenAI’s tool/function-calling contract so you can:

* define an action once (name, description, **parameters/JSON Schema**),
* expose it to the LLM (convert to the API’s `tools` format),
* and execute it when the model returns a tool call.

### Why those params?

* **name** → matches the tool/function `name` the model returns.
* **description** → the short blurb the model sees to decide *when* to use it.
* **parameters** → the **JSON Schema** (or schema source) for arguments; helps the model propose correct args and lets you validate them before running.
* **function** → the actual Python callable you execute.
* **terminal (agent-level)** → tells your controller whether this action ends the loop (e.g., “final\_answer”) vs. an intermediate step.

### Nice pattern: make it provider-agnostic

Keep `Action` as your single source of truth, then adapt to OpenAI (or others) with a tiny converter:

```python
class Action:
    def __init__(self, name, function, description, parameters, terminal=False, validator=None):
        self.name = name
        self.function = function
        self.description = description
        self.parameters = parameters          # JSON Schema dict OR a Pydantic model class
        self.terminal = terminal
        self.validator = validator            # optional: Pydantic TypeAdapter or callable

    def to_openai_tool(self) -> dict:
        schema = (
            self.parameters.model_json_schema()  # if it's a Pydantic model class
            if hasattr(self.parameters, "model_json_schema")
            else self.parameters                  # already a JSON Schema dict
        )
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parameters": schema,
            },
        }

    def validate_args(self, args: dict) -> dict:
        return self.validator(args) if self.validator else args

    def execute(self, **args):
        return self.function(**args)
```

Then you can register actions:

```python
actions = {
    a.name: a
    for a in [
        Action("read_file", read_file, "Reads a file", {"type":"object","properties":{"filename":{"type":"string"}},"required":["filename"]}),
        # ...
    ]
}
tools = [a.to_openai_tool() for a in actions.values()]  # pass to OpenAI API
```

And dispatch safely when the model calls a tool:

```python
call = msg.tool_calls[0]
name = call.function.name
args = json.loads(call.function.arguments)

action = actions[name]
validated = action.validate_args(args)   # optional: Pydantic etc.
result = action.execute(**validated)
```

### Bottom line

`Action` class mirrors the OpenAI tool schema **by design**. That alignment makes your agent modular, portable (easy to add Anthropic/Gemini adapters), and safer (attach validation to the same object that declares the schema).




# What an Action Registry is

A small in-memory catalog that maps **action names → Action objects**. It’s just a layer in your app to:

* **Declare** all available tools in one place
* **Expose** them to a provider (e.g., OpenAI) in the required format
* **Dispatch** model-requested tool calls to your Python functions
* **Validate / authorize / log** calls consistently

Your class:

```python
class ActionRegistry:
    def __init__(self):
        self.actions = {}

    def register(self, action: Action):
        self.actions[action.name] = action  # add (or replace) an Action

    def get_action(self, name: str):
        return self.actions.get(name)

    def get_actions(self):
        return list(self.actions.values())
```

# Why it’s useful (in practice)

* **Decoupling**: Define actions once; reuse across prompts/models.
* **Provider-agnostic**: Convert the same actions to OpenAI tools, Anthropic tools, etc.
* **Validation & safety**: Central place to attach schemas, whitelists, auth checks.
* **Observability**: One hook to log usage, latency, failures.
* **Testing**: Swap in a fake registry for unit tests.

# How it fits with OpenAI tool calling

1. **Register** your actions:

```python
registry = ActionRegistry()
registry.register(Action("read_file", read_file, "Reads a file", read_file_schema))
registry.register(Action("list_files", list_files, "Lists files", list_files_schema))
```

2. **Expose to OpenAI**:

```python
tools = [a.to_openai_tool() for a in registry.get_actions()]
# pass `tools` in your chat.completions.create(...)
```

3. **Dispatch tool calls**:

```python
def handle_tool_call(call, registry):
    name = call.function.name
    args = json.loads(call.function.arguments)

    action = registry.get_action(name)
    if not action:
        raise ValueError(f"Unknown tool: {name}")

    # optional: validate args here (Pydantic/JSON Schema)
    result = action.execute(**args)
    return result
```

# Helpful enhancements (recommended)

* **Duplicate protection**: raise if `name` already registered.
* **Namespacing & versioning**: e.g., `"files.read@v1"`, `"files.read@v2"`.
* **Access control**: register actions with roles/scopes; enforce at dispatch.
* **Selective exposure**: `registry.list_for_openai(allowed={"read_file","list_files"})`.
* **Thread safety**: if multi-threaded, guard `register` with a lock.
* **Decorator sugar**:

  ```python
  def action(name, description, parameters):
      def wrap(func):
          registry.register(Action(name, func, description, parameters))
          return func
      return wrap
  ```

# Bottom line

* **Not an OpenAI thing**: it’s an application pattern.
* It makes your agent **modular, safe, and portable**: define actions once, validate centrally, adapt to any provider’s “tool” envelope, and dispatch reliably.



### ✅ The `@dataclass` Decorator

The `@` symbol in Python is used for **decorators**. A **decorator** is a special kind of function that *modifies* or *enhances* another function or class.

In this case:

```python
from dataclasses import dataclass

@dataclass(frozen=True)
class Goal:
    priority: int
    name: str
    description: str
```

The `@dataclass` decorator is applied to the `Goal` class.

---

### 🔍 What Does `@dataclass` Do?

`@dataclass` automatically generates special methods for the class:

* `__init__()` — constructor (so you don't have to write it)
* `__repr__()` — human-readable string representation
* `__eq__()` — equality comparisons
* `__hash__()` — if needed
* others depending on parameters

So instead of writing this manually:

```python
class Goal:
    def __init__(self, priority, name, description):
        self.priority = priority
        self.name = name
        self.description = description
```

You just define the fields and let the decorator do the work.

---

### 🔐 What About `frozen=True`?

That makes the class **immutable**, meaning you can’t change its attributes after creation:

```python
goal = Goal(1, "list files", "List all the files in the directory")
goal.priority = 2  # ❌ This will raise an error
```

This is useful when you want your goals to remain stable (unchanged once defined), which is common in agent architecture.

---

### 📌 Why Use It?

Because:

1. It keeps the class definition **short and clean**.
2. It avoids **boilerplate** (like manually writing constructors).
3. Immutability (`frozen=True`) makes the `Goal` objects safer to use in concurrent or repeatable agent systems.




### 🚀 **Why You Should Use `@dataclass` in Production:**

| Benefit                     | Why It Matters in Production                               |
| --------------------------- | ---------------------------------------------------------- |
| **Less boilerplate**        | Reduces repetitive code → faster dev and fewer bugs        |
| **Built-in validation**     | Makes it easier to enforce structure (types, immutability) |
| **Readability**             | Improves code clarity, especially for teams                |
| **Immutability (`frozen`)** | Avoids accidental changes → more predictable systems       |
| **Integration friendly**    | Works well with JSON, APIs, config loading, logging, etc.  |

---

### 🆚 When NOT to Use It

There are *some* cases where `@dataclass` may not be ideal:

* You need full control over `__init__` or `__new__` behavior
* You're doing something dynamic or meta-programmatic
* You want extreme performance optimizations (rare)

---

### 🧠 Bottom Line

In most modern Python code — especially **for agents, tool definitions, configs, schema, or records** — use `@dataclass`:

* It aligns perfectly with structured thinking and schemas
* It matches well with the LLM-agent paradigm where clarity and structure are critical
* It is **production-grade** and widely used in libraries like FastAPI, Pydantic, and others

✅ So yes: **Use `@dataclass` in production unless you have a strong reason not to.**



## Action Class Explained
In this pattern you:

1. define a generic `Action` class (your tool “blueprint”),
2. **instantiate one `Action` per tool**, and
3. register them in an `ActionRegistry` so you can both expose them to the LLM and dispatch calls back to Python.

Here’s a tight, paste-ready setup that shows the full flow:

```python
# --- 1) Action & Registry -----------------------------------------
class Action:
    def __init__(self, name, function, description, parameters, terminal=False, validator=None):
        self.name = name
        self.function = function
        self.description = description
        self.parameters = parameters          # JSON Schema dict (or a Pydantic model)
        self.terminal = terminal              # optional: marks “final answer” actions
        self.validator = validator            # optional: args validator

    def to_openai_tool(self) -> dict:
        schema = (
            self.parameters.model_json_schema()
            if hasattr(self.parameters, "model_json_schema")
            else self.parameters
        )
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parameters": schema,
            },
        }

    def validate_args(self, args: dict) -> dict:
        return self.validator(args) if self.validator else args

    def execute(self, **args):
        return self.function(**args)

class ActionRegistry:
    def __init__(self):
        self.actions = {}

    def register(self, action: Action):
        if action.name in self.actions:
            raise ValueError(f"Action '{action.name}' already registered")
        self.actions[action.name] = action

    def get_action(self, name: str):
        return self.actions.get(name)

    def all_tools_for_openai(self):
        return [a.to_openai_tool() for a in self.actions.values()]
```

```python
# --- 2) Your Python tools (business logic) ------------------------
import os

def list_files():
    return os.listdir('.')

def read_file(filename: str):
    path = os.path.abspath(filename)
    if not os.path.isfile(path):
        return {"error": f"File not found: {filename}"}
    with open(path, "r", encoding="utf-8") as f:
        return {"filename": filename, "content": f.read()}

def search_in_file(file_name: str, search_term: str):
    results = []
    if not os.path.isfile(file_name):
        return {"filename": file_name, "matches": [], "error": "not found"}
    with open(file_name, "r", encoding="utf-8") as f:
        for i, line in enumerate(f, start=1):
            if search_term in line:
                results.append({"line": i, "text": line.rstrip("\n")})
    return {"filename": file_name, "matches": results}
```

```python
# --- 3) Register one Action per tool ------------------------------
registry = ActionRegistry()

registry.register(Action(
    name="list_files",
    function=list_files,
    description="List all files in the current directory.",
    parameters={"type": "object", "properties": {}, "required": []},
))

registry.register(Action(
    name="read_file",
    function=read_file,
    description="Read the contents of a text file.",
    parameters={
        "type": "object",
        "properties": {"filename": {"type": "string", "description": "File to read"}},
        "required": ["filename"]
    },
))

registry.register(Action(
    name="search_in_file",
    function=search_in_file,
    description="Find lines containing a search term in a file.",
    parameters={
        "type": "object",
        "properties": {
            "file_name": {"type": "string"},
            "search_term": {"type": "string"}
        },
        "required": ["file_name", "search_term"]
    },
))
```

```python
# --- 4) Expose to OpenAI & dispatch tool calls --------------------
tools = registry.all_tools_for_openai()   # pass this in your API call

def dispatch_tool_call(call, registry: ActionRegistry):
    import json
    name = call.function.name
    args = json.loads(call.function.arguments)

    action = registry.get_action(name)
    if not action:
        return {"error": f"Unknown tool '{name}'"}

    # optional: validate args (Pydantic/JSON Schema)
    validated = action.validate_args(args)
    return action.execute(**validated)
```

### Why this is nice

* **One Action per tool** keeps things modular and testable.
* The registry is your **single source of truth**: you generate the OpenAI `tools` payload from it and also use it to **dispatch** calls safely.
* You can attach **validation**, auth checks, logging, versioning, or a `terminal` flag to each action without scattering logic.

So yes—your understanding is correct: you create the `Action` class once, then **instantiate it for each tool** you want the agent to use, and register them with the `ActionRegistry`.



* Your **Python functions** (`read_file`, `search_in_file`) are the *implementations*.
* Your **`Action` objects** hold the *metadata + schema* (name/description/**parameters**) about those functions.
* The **OpenAI API** doesn’t see your Python functions directly; it only sees a **`tools` list** you send, built **from your `Action` objects**.
* At runtime, the **LLM** returns **arguments** (JSON) for a tool call that *match the schema* you put in `Action.parameters`. Those args are not “in” the `Action` ahead of time—they’re produced by the model when it decides to call the tool.

### What you’re missing: convert `Action` → OpenAI `tools`

Add a helper to your `Action` class and build the `tools` array from your registry:

```python
class Action:
    def __init__(self, name, function, description, parameters, terminal=False):
        self.name = name
        self.function = function
        self.description = description
        self.terminal = terminal
        self.parameters = parameters  # JSON Schema dict

    def to_openai_tool(self) -> dict:
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parameters": self.parameters,
            },
        }

    def execute(self, **args):
        return self.function(**args)
```

Register your actions **with schemas that match your function parameter names** (important for `**args`):

```python
registry = ActionRegistry()

registry.register(Action(
    name="list_files",
    function=list_files,
    description="List all files in the current directory",
    parameters={"type": "object", "properties": {}, "required": []},
))

registry.register(Action(
    name="read_file",
    function=read_file,
    description="Read the contents of a text file",
    parameters={
        "type": "object",
        "properties": {"file_name": {"type": "string"}},
        "required": ["file_name"],
    },
))

registry.register(Action(
    name="search_in_file",
    function=search_in_file,
    description="Find lines containing a term in a file",
    parameters={
        "type": "object",
        "properties": {
            "file_name": {"type": "string"},
            "search_term": {"type": "string"},
        },
        "required": ["file_name", "search_term"],
    },
))
```

Build the `tools` payload you pass to OpenAI:

```python
tools = [a.to_openai_tool() for a in registry.get_actions()]
```

### How the args “show up”

1. You send `tools=...` (from your `Action`s) in the API call.
2. The model may respond with a tool call like:

   ```json
   {
     "tool_calls":[
       {"function":{"name":"read_file","arguments":"{\"file_name\":\"notes.txt\"}"}}
     ]
   }
   ```
3. You parse and **dispatch** via the registry:

```python
call = msg.tool_calls[0]
action = registry.get_action(call.function.name)
args = json.loads(call.function.arguments)   # {"file_name": "notes.txt"}
result = action.execute(**args)              # calls your Python function
```

### Key gotchas to avoid

* **Schema keys must match function kwargs** if you use `execute(**args)`. (You have `file_name` in code—make sure the schema uses `file_name`, not `filename`.)
* `Action.parameters` is the **schema**, not the runtime arguments.
* The **runtime arguments are generated by the LLM** to satisfy your schema and arrive in `message.tool_calls[*].function.arguments`.

Once you add `to_openai_tool()` and build `tools` from the registry, everything clicks: `Action` defines the tool, the registry catalogs it, the API sees the schema, and the LLM supplies the args at call time.
